In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from glob import glob
import seaborn as sns
from PIL import Image

np.random.seed(42)

from tensorflow import keras
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import LabelEncoder

import autokeras as ak

In [ ]:
tumor_df = pd.read_csv("bt_dataset_t3.csv")

In [ ]:
SIZE = 64
print(tumor_df.sample(10))

In [ ]:
imagesList = []
labelsList = []

for index, row in tumor_df.iterrows():
    image_path= "lib/Brain Tumor/Brain Tumor/"+row['Image']+".jpg"
    img = np.asarray(Image.open(image_path).resize((SIZE, SIZE)))

    imagesList.append(img)

    labelsList.append(row['Target'])

X = np.array(imagesList)
Y = np.array(labelsList)

In [ ]:
print(X.shape)
print(Y)

In [ ]:
X = X/255


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
print(x_train.shape, y_train.shape)


In [ ]:
#Autokeras that was used to determine the best model
#classifier = ak.ImageClassifier(max_trials=4)
#classifier.fit(x_train, y_train, verbose=1, epochs= 10)
#best_model = classifier.export_model()
#best_model.summary()

In [ ]:
model = Sequential([
    
    #------Input layer-------
    tf.keras.layers.InputLayer(input_shape=(64, 64, 3)),

    #------First layer-------
    #
    Conv2D(32, (3, 3), activation ='relu', padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    #------Second layer-------
    Conv2D(64, (3, 3), activation='relu', padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Dropout(0.25),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')

])

In [ ]:
#Compile
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

#Structure of the model
model.summary()

In [ ]:
history = model.fit(x_train, y_train, 
                    validation_data=(x_test, y_test), 
                    epochs=20, batch_size=32, verbose=1)